# Обработка и дополнение данных

---

### Цели

1. Подготовить датасеты для моделей обучения
2. 

In [1]:
import json
import os
from multiprocessing import Pool
import sys
import matplotlib.pyplot as plt
import pandas as pd
import pprint
import progressbar
import re
import requests
import questionary
import seaborn as sns
import time
import urllib
import warnings

In [2]:
plt.style.use('ggplot')
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 50
pd.options.display.max_columns = 20

In [3]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

### Категории

In [ ]:
df_news.describe(include=[object]).T

небольшое количество записей содержит информацию в колонках preview, text, images. При этом, у нас есть колонки со схожими названиями: preview_text, full_text, image.

In [ ]:
# проверим несколько записей с пустыми значениями в столбце preview
df_news[df_news.preview.isna()][['preview', 'preview_text', 'text', 'full_text', 'image', 'images']][:5]

In [ ]:
# проверим несколько записей с пустыми значениями в столбце preview_text
df_news[df_news.preview_text.isna()][['preview', 'preview_text', 'text', 'full_text', 'image', 'images']][:5]

In [ ]:
string_columns = df_news.select_dtypes(object).columns

In [ ]:
string_columns

In [ ]:
val_columns = df_news.select_dtypes(exclude=object).columns

In [ ]:
val_columns

In [ ]:
df_news[string_columns] = df_news[string_columns].fillna('')

In [ ]:
df_news[val_columns] = df_news[val_columns].fillna(0)

In [ ]:
df_news

 MEMO: перенести данные в одну колонку: preview -> preview_text, text -> full_text, images -> image

In [ ]:
df_news.preview_text = df_news.apply(lambda c: c.preview if len(c.preview) > 0 else c.preview_text, axis=1)

In [ ]:
df_news.full_text = df_news.apply(lambda c: c.text if len(c.text) > 0 else c.full_text, axis=1)

In [ ]:
def get_text_on_pattern_replacement_func(html_text: str) -> str:
    patterns_to_replace = {
        '&nbsp;': ' ',  # замена пробела(&nbsp;) на пробел
        '<.+?>|\\n|&[a-z]+;': '',  # замена всех тэгов, /n и выражений типа &ldquo; на пустую строку
        '\s+': ' ',  # замена всех повторяющихся пробелов на один
        '(.\.)(\S)': '\g<1> \g<2>',
        # добавление пробела после конца предложения, если дальше нет пробела и начинается буква
        '\s+$': '',  # удаление пробелов с конца строки
    }

    for pattern, repl in patterns_to_replace.items():
        html_text = re.sub(pattern, repl, html_text)
        return html_text

In [ ]:
# df_news['full_text_cleaned'] = df_news.apply(lambda c: get_text_on_pattern_replacement_func(c.full_text), axis=1)

In [ ]:
# df_news['full_text_cleaned'] = [get_text_on_pattern_replacement_func(i) for i in df_news.full_text.values]

### Значения, даты, булеаны

In [ ]:
df_news.describe(exclude=[object]).T

### Проверка наличия необходимых полей в датасетах

In [ ]:
# dataset['news_id'] = dataset.apply(lambda i: i.url_clean.split('/')[-2], axis=1)

In [ ]:
dataset['news_id'] = dataset.url_clean.str.extract(r"(\d{7})")

In [ ]:
dataset[dataset.news_id.isna()]

In [ ]:
dataset.news_id.fillna(0, inplace=True)

In [ ]:
dataset.news_id = [int(i) for i in dataset.news_id]

In [ ]:
df_news[~df_news.id.isin(dataset.news_id.unique())][['id', 'title', 'preview_text', 'created_at', 'published_at', 'updated_at']]

##### бОльшая часть данных в предоставленном датасете не имеет соответствующей новости... 

In [ ]:
dataset.iloc[13219]

## Get full data from api

#### Articles

In [ ]:
articles_api = 'https://www.mos.ru/api/newsfeed/v4/frontend/json/ru/articles?expand=spheres,kind,image&fields=id,title,importance,published_at,created_at,updated_at,is_deferred_publication,status,ya_rss,active_from,active_to,oiv_id,search,display_image,label,icon_id,canonical_url,canonical_updated_at,is_powered,has_image,date,has_district,date_timestamp,tags,theme_id,theme_ids,themes,spheres,sphere,kind,is_oiv_publication,organizations,updated_at_timestamp,created_at_timestamp,attach,active_from_timestamp,active_to_timestamp,image,counter,territory_area_id,territory_district_id,preview_text,full_text,url,preview,text,promo,images'

In [ ]:
def get_api_data(base_api_url, filename='article_full.json') -> json:
    response = requests.get(base_api_url)
    data = response.json()
    n_pages = data.get("_meta").get("pageCount") + 1
    print("Total number of pages: ", n_pages)
    full_data = []
    pbar = progressbar.ProgressBar(widgets=[
                          progressbar.SimpleProgress(),
                          progressbar.Bar(),
                          progressbar.ETA(),
                          progressbar.FileTransferSpeed(),
                          ],
                          max_value=n_pages).start()
    for i in range(n_pages):
        page = requests.get(base_api_url + f"&page={i}")
        try:
            full_data.append(page.json().get('items'))
        except Exception as e:
            print(f"Sorry, man! We got {e}")
#             questionary.select("Try to limit number of fields and options to expand:",
#                                choices=[
#                                    "spheres",
#                                    "kind",
#                                    "image",
#                                    "preview",
#                                    "text",
#                                    "promo",
#                                    "images",
#                                ]
#                               ).ask()
#             page = requests.get(base_api_url[:62] + f"&page={i}")
#             full_data.append(page.json().get('items')[0])
        pbar.update(i+1)
#         print(f"page {i}: ", len(page.json().get("items", "error")))
    with open(os.path.join(BASE_DIR, "data", filename), "w") as file:
        json.dump(full_data, file)
        pbar.finish()

In [ ]:
get_api_data(articles_api, 'articles_full.json')

##### Тут была попытка собрать всё побыстрее, используя мультипроцессинг и map... пока безуспешно. Работает на несколько порядков быстрее, чем обычный перебор страниц выше, но данные в финальном датасете только по первой и последней странице. Кто-нибудь сталкивался с подобным?

In [ ]:
# full_articles = []

In [ ]:
# for i in range(1, n_pages + 1):
#     r = requests.get(articles_api+f'&page={i}')
#     data = r.json()
#     full_articles.append(data.get('items'))

In [ ]:
# def req(page):
#     url = articles_api+f'&page={page}'
#     r = requests.get(url)
#     data = r.json()
#     return data

In [ ]:
# with Pool(processes = 5) as P:
#     results = P.map(req, list(range(1, n_pages + 1)))

In [ ]:
# results[2]

In [ ]:
# flat_results = [item for sublist in results for item in sublist]

In [ ]:
# with open(os.path.join(BASE_DIR, 'data', 'articles_full.json'), 'w') as file:
#     json.dump(flat_results, file)

#### News

In [ ]:
news_api = 'https://www.mos.ru/api/newsfeed/v4/frontend/json/ru/news?expand=spheres,kind,image,tags&fields=id,title,importance,published_at,created_at,updated_at,is_deferred_publication,status,ya_rss,active_from,active_to,oiv_id,search,display_image,label,icon_id,canonical_url,canonical_updated_at,is_powered,has_image,date,has_district,date_timestamp,tags,theme_id,theme_ids,themes,spheres,sphere,kind,is_oiv_publication,organizations,updated_at_timestamp,created_at_timestamp,attach,active_from_timestamp,active_to_timestamp,image,counter,territory_area_id,territory_district_id,preview,text,url,promo,images'

In [ ]:
get_api_data(news_api, 'news_full.json')

In [ ]:
# news_api = 'https://www.mos.ru/api/newsfeed/v4/frontend/json/ru/news?expand=spheres,kind,image,tags,attach&fields=id,title,importance,published_at,created_at,updated_at,is_deferred_publication,status,ya_rss,active_from,active_to,oiv_id,search,display_image,label,icon_id,canonical_url,canonical_updated_at,is_powered,has_image,date,has_district,date_timestamp,tags,theme_id,theme_ids,themes,spheres,sphere,kind,is_oiv_publication,organizations,updated_at_timestamp,created_at_timestamp,attach,active_from_timestamp,active_to_timestamp,image,counter,territory_area_id,territory_district_id,preview_text,full_text,url,preview,text,promo,images'

In [ ]:
# r = requests.get(news_api)

In [ ]:
# data = r.json()

In [ ]:
# n_pages = data.get('_meta').get('pageCount')

In [ ]:
# def get_urls(n_pages):
#     for i in range(1, n_pages + 1):
#         url = news_api+f'&page={i}'
#         yield url

In [ ]:
# urls = [news_api+f'&page={i}' for i in range(1, n_pages + 1)]

In [ ]:
# def req(url):
#     r = requests.get(url)
#     time.sleep(0.2)
#     data = r.json()
# #     print('getting url: ', url)
# #     print('received data: ', data.get('items', 'error'))
#     return data

In [ ]:
# full_news = []

In [ ]:
# with Pool(processes=5) as P:
#     results = P.map(req, get_urls(n_pages))
#     for _ in tqdm.tqdm(P.imap_unordered(req, urls), total=len(urls)):
#         full_news.append(_)
#     results = list(tqdm.tqdm(P.imap_unordered(req, urls), total=len(urls)))
#     full_news.append(results)

In [ ]:
# full_news[-100:]
# results[-100:]

In [ ]:
# full_news = []
# for i in range(1, n_pages + 1):
#     r = requests.get(news_api+f'&page={i}')
#     data = r.json()
#     full_news.append(data.get('items'))

In [ ]:
# flat_results = [item for sublist in results for item in sublist]

In [ ]:
# with open(os.path.join(BASE_DIR, 'data', 'news_full.json'), 'w') as file:
#     json.dump(flat_results, file)

In [ ]:
# from alive_progress import alive_bar
# from time import sleep

# with alive_bar(100) as bar:   # default setting
#     for i in range(100):
#         sleep(0.03)
#         bar()                        # call after consuming one item

# using bubble bar and notes spinner
# with alive_bar(200, bar = 'bubbles', spinner = 'notes2') as bar:
#     for i in range(200):
#         sleep(0.03)
#         bar()                    # call after consuming one item

In [ ]:
# with open(os.path.join(BASE_DIR, 'data', 'news_full.json'), 'a'):
#     json.dumps(data)

In [ ]:
# urllib.parse.unquote(data.get('_links').get('next').get('href'))

In [ ]:
with open(os.path.join(BASE_DIR, 'data', 'news_full.json')) as file:
    data = json.load(file)